In [1]:
!rm -f minotaur-0.4.1-linux-x86_64.tar.gz
!wget -nc https://www.ieor.iitb.ac.in/files/faculty/amahajan/minotaur/bin/minotaur-0.4.1-linux-x86_64.tar.gz
!tar -zxf minotaur-0.4.1-linux-x86_64.tar.gz
!rm -f mbin
!ln -s minotaur-0.4.1-linux-x86_64/bin mbin


--2025-08-12 15:11:02--  https://www.ieor.iitb.ac.in/files/faculty/amahajan/minotaur/bin/minotaur-0.4.1-linux-x86_64.tar.gz
Resolving www.ieor.iitb.ac.in (www.ieor.iitb.ac.in)... 103.21.125.143
Connecting to www.ieor.iitb.ac.in (www.ieor.iitb.ac.in)|103.21.125.143|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23422040 (22M) [application/x-gzip]
Saving to: ‘minotaur-0.4.1-linux-x86_64.tar.gz’

minotaur-0.4.1-linu 100%[===================>]  22.34M  10.8MB/s    in 2.1s    

2025-08-12 15:11:05 (10.8 MB/s) - ‘minotaur-0.4.1-linux-x86_64.tar.gz’ saved [23422040/23422040]



In [ ]:
# !pip install -i https://pypi.gurobi.com gurobipy

In [2]:
pip install pyomo

In [3]:
from pyomo.environ import *

In [ ]:
# !apt-get install -y -qq coinor-libipopt-dev

#



In [ ]:
# from pyomo.environ import SolverFactory

# # Check if IPOPT solver is available
# solver = SolverFactory('ipopt')
# solver_available = solver.available()
# print(f"IPOPT Available: {solver_available}")


In [ ]:
# import os
# os.environ['PATH'] += os.pathsep + os.getcwd() + '/ipopt-linux64/bin'



In [4]:
from scipy.stats import norm

In [5]:
from pyomo.environ import *  # Import Pyomo first
import random
 # Import scipy after Pyomo


In [6]:
import random
from scipy.stats import norm  # For statistical norm functions
from pyomo.environ import *  # Ensure Pyomo is imported

# Problem Parameters
num_slots = 10
probability = 0.1
arrival_stream = ['B', 'B', 'B', 'P', 'B', 'A', 'A', 'A', 'A', 'A', 'P', 'A', 'A', 'P', 'P',
                  'A', 'A', 'A', 'B', 'B', 'A', 'B', 'A', 'B', 'B', 'A', 'P', 'P', 'B', 'B']
mean = {'A': 36.596, 'B': 44.823, 'P': 56.807}
sigma = {'A': 15.60, 'B': 11.65, 'P': 18.23}

mu = [mean[patient] for patient in arrival_stream]
sigmas = [sigma[patient] for patient in arrival_stream]

# Create a Concrete Model
model = ConcreteModel()

# Define the set of patients and slots
model.I = RangeSet(len(arrival_stream))
model.J = RangeSet(1, num_slots)

# Define decision variables
model.x = Var(model.I, model.J, within=Binary)  # Binary
model.alpha = Var(model.J, within=NonNegativeReals)

# Define the parameter 'c'
c = norm.ppf(probability)

# Objective function
model.obj = Objective(expr = sum((num_slots - j + 1) * sum(mu[i-1] * model.x[i, j] for i in model.I) for j in model.J) - sum(j * 120 for j in model.J), sense=minimize)

# Slot assignment constraints: each patient should be assigned to exactly one slot
model.slot_assignment_constraint = ConstraintList()
for i in model.I:
    model.slot_assignment_constraint.add(sum(model.x[i, j] for j in model.J) == 1)

model.slots_constraint = ConstraintList()
for j in model.J:
    model.slots_constraint.add(sum(mu[i-1] * model.x[i, k] for i in model.I for k in range(1, j+1)) + c * model.alpha[j] >= j * 120)

from pyomo.environ import sqrt

# Modify the second-order cone constraint using the square root
model.slot_variance_constraint = ConstraintList()
for j in model.J:
    lhs = sum((sigmas[i-1] * model.x[i, k])**2 for i in model.I for k in range(1, j+1))
    # Take square root on both sides
    model.slot_variance_constraint.add(sqrt(lhs) <= model.alpha[j])




In [ ]:
mntr = SolverFactory("mbnb", executable='/content/mbin/mbnb')

mntr.options['--log_level'] = 3
mntr.options['--time_limit'] = 600
mntr.options['--qp_engine'] = 'None'
mntr.options['--nlp_engine'] = 'IPOPT'

# Set tee to True if you want to see solver output
result = mntr.solve(model, tee=True)

Environment: User provided options:
  problem_file = /tmp/tmpqmz50x4w.pyomo.nl
  ampl = 1
  log_level = 3
  time_limit = 600
  qp_engine = None
  nlp_engine = IPOPT
Environment: End of user provided options.

Bnb: Minotaur version 0.4 patch 1 git revision v0.4-50-g456fd8cc
Bnb: NLP based Branch-and-bound algorithm for convex MINLP
Environment: User provided options:
  log_level = 3
  time_limit = 600
  qp_engine = None
  nlp_engine = IPOPT
Environment: End of user provided options.

AMPLInterface: problem type is MINLP
Solver: time used in reading instance = 0.01
Bnb: objective sense: minimize
Bnb: handlers used in presolve:
Bnb: LinearHandler (Handling linear constraints)
Bnb: NlPresHandler (presolving nonlinear constraints).
Presolver: Presolving ... 
Presolver: Finished presolving.
LinearHandler: Statistics for presolve by Linear Handler:
LinearHandler: Number of iterations           = 2
LinearHandler: Time taken in initial presolve = 0.00
LinearHandler: Time taken in node presolves

  - termination condition: maxIterations
  - message from solver: 


In [ ]:
print("Solver termination status:", result.solver.status)
print("Solver termination condition:", result.solver.termination_condition)
print("Best solution value:", model.obj())
print("Best bound: ", result.problem.lower_bound)
print("Solver time:", result.solver.time)
# print("Solver time:", rmodel.obj())

Solver termination status: warning
Solver termination condition: maxIterations
Best solution value: 867.2862358044031
Best bound:  -inf
Solver time: 616.8592071533203


In [ ]:
for i in model.I:
  for j in model.J:
    print(f'for x{[i,j]}',model.x[i,j].value)

for x[1, 1] 0.9999999902819566
for x[1, 2] -9.417036297765528e-09
for x[1, 3] 7.737214242460948e-09
for x[1, 4] 1.0553807510645331e-08
for x[1, 5] 1.0756798658333862e-08
for x[1, 6] 1.0832722973218002e-08
for x[1, 7] 1.0872501706661256e-08
for x[1, 8] 1.0896991230459302e-08
for x[1, 9] 1.0913587106542408e-08
for x[1, 10] 1.0925576686669788e-08
for x[2, 1] 0.9999999902819571
for x[2, 2] -9.417031587718447e-09
for x[2, 3] 7.737008072668491e-09
for x[2, 4] 1.055380732108581e-08
for x[2, 5] 1.0756798687386897e-08
for x[2, 6] 1.0832723001682013e-08
for x[2, 7] 1.0872501727650424e-08
for x[2, 8] 1.089699124594641e-08
for x[2, 9] 1.0913587118293068e-08
for x[2, 10] 1.0925576695842495e-08
for x[3, 1] -9.718225735487572e-09
for x[3, 2] -9.41896480460189e-09
for x[3, 3] 7.819637812853067e-09
for x[3, 4] 1.0000000105539266
for x[3, 5] 1.0756797865738007e-08
for x[3, 6] 1.0832716292764387e-08
for x[3, 7] 1.0872495909705556e-08
for x[3, 8] 1.0896986676052169e-08
for x[3, 9] 1.0913583533554415e-08
f

In [ ]:
for i in model.I:
    for j in model.J:
        # Check if x[i, j] is close to 1, indicating an assignment
        if abs(model.x[i, j].value - 1) < 1e-6:  # Adjust the tolerance as needed
            print(f'Patient {i} is assigned to Slot {j}')


Patient 1 is assigned to Slot 1
Patient 2 is assigned to Slot 1
Patient 3 is assigned to Slot 4
Patient 4 is assigned to Slot 5
Patient 5 is assigned to Slot 3
Patient 6 is assigned to Slot 6
Patient 7 is assigned to Slot 1
Patient 8 is assigned to Slot 8
Patient 9 is assigned to Slot 2
Patient 10 is assigned to Slot 10
Patient 11 is assigned to Slot 2
Patient 12 is assigned to Slot 10
Patient 13 is assigned to Slot 8
Patient 14 is assigned to Slot 7
Patient 15 is assigned to Slot 6
Patient 16 is assigned to Slot 6
Patient 17 is assigned to Slot 1
Patient 18 is assigned to Slot 9
Patient 19 is assigned to Slot 3
Patient 20 is assigned to Slot 7
Patient 21 is assigned to Slot 9
Patient 22 is assigned to Slot 8
Patient 23 is assigned to Slot 4
Patient 24 is assigned to Slot 7
Patient 25 is assigned to Slot 10
Patient 26 is assigned to Slot 2
Patient 27 is assigned to Slot 4
Patient 28 is assigned to Slot 5
Patient 29 is assigned to Slot 9
Patient 30 is assigned to Slot 3


In [ ]:
# Initialize a dictionary to store patients for each slot
slot_assignments = {j: [] for j in model.J}

# Loop through the assignments and collect patients for each slot
for i in model.I:
    for j in model.J:
        if abs(model.x[i, j].value - 1) < 1e-6:  # Adjust tolerance if needed
            slot_assignments[j].append(i)

# Print the assignments in the desired format
for slot, patients in slot_assignments.items():
    print(f'Slot {slot} -> {patients}')


Slot 1 -> [1, 2, 7, 17]
Slot 2 -> [9, 11, 26]
Slot 3 -> [5, 19, 30]
Slot 4 -> [3, 23, 27]
Slot 5 -> [4, 28]
Slot 6 -> [6, 15, 16]
Slot 7 -> [14, 20, 24]
Slot 8 -> [8, 13, 22]
Slot 9 -> [18, 21, 29]
Slot 10 -> [10, 12, 25]


#Smothing the not differentiable constrain

In [ ]:
import random
from scipy.stats import norm  # For statistical norm functions
from pyomo.environ import *  # Ensure Pyomo is imported
import math  # For logarithmic and exponential functions

# Problem Parameters
num_slots = 10
probability = 0.1
arrival_stream = ['B', 'B', 'B', 'P', 'B', 'A', 'A', 'A', 'A', 'A', 'P', 'A', 'A', 'P', 'P',
                  'A', 'A', 'A', 'B', 'B', 'A', 'B', 'A', 'B', 'B', 'A', 'P', 'P', 'B', 'B']
mean = {'A': 36.596, 'B': 44.823, 'P': 56.807}
sigma = {'A': 15.60, 'B': 11.65, 'P': 18.23}

mu = [mean[patient] for patient in arrival_stream]
sigmas = [sigma[patient] for patient in arrival_stream]

# Create a Concrete Model
model = ConcreteModel()

# Define the set of patients and slots
model.I = RangeSet(len(arrival_stream))
model.J = RangeSet(1, num_slots)

# Define decision variables
model.x = Var(model.I, model.J, within=Binary)  # Binary
model.alpha = Var(model.J, within=NonNegativeReals)

# Define the parameter 'c' for probabilistic constraint
c = norm.ppf(probability)

# Objective function
model.obj = Objective(
    expr=sum((num_slots - j + 1) * sum(mu[i-1] * model.x[i, j] for i in model.I) for j in model.J)
         - sum(j * 120 for j in model.J),
    sense=minimize
)

# Slot assignment constraints: each patient should be assigned to exactly one slot
model.slot_assignment_constraint = ConstraintList()
for i in model.I:
    model.slot_assignment_constraint.add(sum(model.x[i, j] for j in model.J) == 1)

# Variance constraint with Log-Sum-Exp approximation
beta = 2  # Smoothing parameter, adjust as needed
model.slot_variance_constraint = ConstraintList()
for j in model.J:
    # Log-Sum-Exp approximation for the L2 norm
    log_sum_exp_expr = sum(exp((sigmas[i-1] * model.x[i, k])**2 / beta) for i in model.I for k in range(1, j+1))
    model.slot_variance_constraint.add(log(log_sum_exp_expr) <= model.alpha[j])

# Slots constraint for total time requirement with probabilistic adjustment
model.slots_constraint = ConstraintList()
for j in model.J:
    model.slots_constraint.add(sum(mu[i-1] * model.x[i, k] for i in model.I for k in range(1, j+1)) + c * model.alpha[j] >= j * 120)



AttributeError: '_PyDrive2ImportHook' object has no attribute 'find_spec'

In [ ]:
mntr = SolverFactory("mbnb", executable='/content/mbin/mbnb')

mntr.options['--log_level'] = 3
mntr.options['--time_limit'] = 6000
mntr.options['--qp_engine'] = 'None'
mntr.options['--nlp_engine'] = 'IPOPT'

# Set tee to True if you want to see solver output
result = mntr.solve(model, tee=True)

Environment: User provided options:
  problem_file = /tmp/tmp53frmfhm.pyomo.nl
  ampl = 1
  log_level = 3
  time_limit = 6000
  qp_engine = None
  nlp_engine = IPOPT
Environment: End of user provided options.

Bnb: Minotaur version 0.4 patch 1 git revision v0.4-50-g456fd8cc
Bnb: NLP based Branch-and-bound algorithm for convex MINLP
Environment: User provided options:
  log_level = 3
  time_limit = 6000
  qp_engine = None
  nlp_engine = IPOPT
Environment: End of user provided options.

AMPLInterface: problem type is MINLP
Solver: time used in reading instance = 0.01
Bnb: objective sense: minimize
Bnb: handlers used in presolve:
Bnb: LinearHandler (Handling linear constraints)
Bnb: NlPresHandler (presolving nonlinear constraints).
Presolver: Presolving ... 
Presolver: Finished presolving.
LinearHandler: Statistics for presolve by Linear Handler:
LinearHandler: Number of iterations           = 2
LinearHandler: Time taken in initial presolve = 0.00
LinearHandler: Time taken in node presolv

  - termination condition: maxIterations
  - message from solver: 
